# Reading PPPC4DMID Tables with gammalib

This nb is to show only how the projected Spectral-DM class should work.

Reading PPPC4DMID (Cirelli *et al.*, 2011) .dat files and converting to gammalib Fits Tables

In [1]:
import gammalib

Reading the file for spectra of dm annihilations with gamma-ray productions in the final states and with EW corrections

In [2]:
ffile = '../ctaAnalysis/data/AtProduction_gammas.dat'    #    You can change the path here :P
table = gammalib.GCsv( ffile )

In [3]:
print( table )

=== GCsv ===
 Number of columns .........: 30
 Number of rows ............: 11099
 Floating point precision ..: default


So, PPPC4DMID tables contains the mass of DM candidate in the first columns, and the value of $\log{x}$ in the second column, where $x$ is:

$x = \frac{E_{\gamma}}{m_{\text{DM}}}$

The rest of the columns in the PPPC4DMID tables contains the differential spectra:

$\frac{\mathrm{d}N}{\mathrm{d}\log{x}}$

I want to extract the unique values for DM masses and $\log{x}$, and save these values in a GFitsCol

In [4]:
def unique_elements( mylist ) :
    
    unique = []
    
    for x in mylist :
        if x not in unique :
            unique.append( x )
            
    return unique

dmmasses = []    #    List to save values of masses
logxvals = []    #    List to save values of logx

for row in range( table.nrows() ) :

    if row > 0 :
        
        dmmasses.append( table.integer( row , 0 ) )
        logxvals.append( table.real( row , 1 ) )

#    Unique values for mass and logx
dmmunique = unique_elements( dmmasses )
logx      = unique_elements( logxvals )


I do the same for the channels

In [5]:
#    List to save annihilation channels
channels = []

for col in range( table.ncols() ) :
    
    if col > 1 :
        
        ch = table.string( 0 , col )
        #    Remove some junk chars :P
        channels.append( ch.translate( { ord( k ): None for k in '\\\[\]'} ) )


## Saving to GFitsTable. I

Now, I am saving the relevant values for masses, $x$ and annihilation channels.

First for masses:

In [6]:
masses = gammalib.GFitsTableLongCol( 'DMMasses' , len( dmmunique ) )

for index in range( len( dmmunique ) ) :
    
    masses[ index ] = dmmunique[ index ]

#    Unit of masses
masses.unit( 'GeV' )

#    Create a BinTable to save column
mtable = gammalib.GFitsBinTable()

mtable.append( masses )
#    Add name to the binTable
mtable.extname( 'DM-Masses' )

print( mtable )

=== GFitsTable ===
 HDU number ................: 0
 HDU name ..................: DM-Masses
 Table type ................: Binary table
 Number of rows ............: 62
 Number of columns .........: 1
=== GFitsHeader (7 cards) ===


And for channels and $x$ (I will convert from $\log{x}$ to ${x}$)

In [7]:
gchannels = gammalib.GFitsTableStringCol( 'Channel' , len( channels ) , 7 )

for i , channel in enumerate( channels ) :
    gchannels[ i ] = channel

#    then, create binary tables for the channels
chtable = gammalib.GFitsBinTable()

chtable.append( gchannels )
chtable.extname( 'Channels' )

print( chtable )

#    The same for xvalues
xvals = gammalib.GFitsTableDoubleCol( 'xValues' , len( logx ) )

for i, xval in enumerate( logx ) :
    xvals[ i ] = 10**xval

xvals.unit( 'Dimensionless (E/m_DM)' )

xtable = gammalib.GFitsBinTable()

xtable.extname( 'E/m_DM' )
xtable.append( xvals )

print( xtable )

=== GFitsTable ===
 HDU number ................: 0
 HDU name ..................: Channels
 Table type ................: Binary table
 Number of rows ............: 28
 Number of columns .........: 1
=== GFitsHeader (7 cards) ===
=== GFitsTable ===
 HDU number ................: 0
 HDU name ..................: E/m_DM
 Table type ................: Binary table
 Number of rows ............: 179
 Number of columns .........: 1
=== GFitsHeader (7 cards) ===


## Getting spectra and saving to a GFitsTable

The following functions get:

1. Indices where the PPC4DMID first column is equal to some DM mass
2. Index for channel in the PPPC4DMID header
3. The $\mathrm{d}N/\mathrm{d}{x}$ for specific channel and mass

In [8]:
#    get index where column is equal to mass in PPPC4DMID table
def get_indices( mass , table ) :
    
    indices = []
    
    for row in range( table.nrows() ) :
        
        if row > 0 :
            if table.integer( row , 0 ) == mass :
                indices.append( row )

    return indices


In [9]:
#    Get index for channel :P
def get_index( ch , channels ) :

    index = 0
    
    for row in range( channels.nrows() ) :
        
        if ch == channels.string( row ) :
            index = row
            
    return index


In [10]:
#    Get dn/dlogx for a set masses in the PPPC4DMID and channel ch
def create_spectra_col( masses , xvals , ch , gchannels , table ) :
    
    nrows = masses.nrows()
    size  = xvals.nrows()
    
    spectra_col  = gammalib.GFitsTableDoubleCol( ch , nrows , size )
    channel_col  = get_index( ch , gchannels ) + 2
    
    for row in range( nrows ) :
        
        spec_indices = get_indices( masses.integer( row ) , table )
        
        for index in range( size ) :
            
            dndlogx = table.real( spec_indices[ index ], channel_col )
            x       = xvals[ index ]
            #    Convert from dn/dlogx to dn/dx
            spectra_col[ row , index ] = dndlogx / x / gammalib.ln10
    
    return spectra_col


Now, I create a table to save the spectra for all channels

In [11]:
#    Now getting all the spectra for the channels and save them into a fits table

#    Create bin table to save spectra data
dmtable = gammalib.GFitsBinTable()

#    I will save all the channels in PPPC4DMID tables
#    So, I need to loop over al the channels
for ich in range( gchannels.nrows() ) :
    
    channel = gchannels[ ich ]
    thiscol = create_spectra_col( masses , xvals , channel , gchannels , table )
    dmtable.append( thiscol )

#    Add cards and extension
dmtable.extname( 'DMSpectra' )

dmtable.card( 'dn/dx' , 'PPPC4DMID' , 'EWCorrections' )

print( dmtable )


=== GFitsTable ===
 HDU number ................: 0
 HDU name ..................: DMSpectra
 Table type ................: Binary table
 Number of rows ............: 62
 Number of columns .........: 28
=== GFitsHeader (89 cards) ===


## Saving to fits file

Finally, create a fits file and save all the spectra

In [12]:
#     And now, append all the tables
thisfits = gammalib.GFits()
thisfits.append( mtable )
thisfits.append( xtable )
thisfits.append( chtable )
thisfits.append( dmtable )

print( thisfits )

fitsname = 'PPPC4DMIDGammasEW.fits'
clobber  = True    #    Overwrite files
thisfits.saveto( fitsname , clobber )

=== GFits ===
 Filename ..................: 
 History ...................: new file
 Mode ......................: read/write
 Number of HDUs ............: 5
=== GFitsImage ===
 HDU number ................: 0
 HDU name ..................: Primary
 Image type ................: unsigned byte
 Number of dimensions ......: 0
 Number of image pixels ....: 0
=== GFitsTable ===
 HDU number ................: 1
 HDU name ..................: DM-Masses
 Table type ................: Binary table
 Number of rows ............: 62
 Number of columns .........: 1
=== GFitsTable ===
 HDU number ................: 2
 HDU name ..................: E/m_DM
 Table type ................: Binary table
 Number of rows ............: 179
 Number of columns .........: 1
=== GFitsTable ===
 HDU number ................: 3
 HDU name ..................: Channels
 Table type ................: Binary table
 Number of rows ............: 28
 Number of columns .........: 1
=== GFitsTable ===
 HDU number ................: 4
 